In [110]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [111]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

df = pd.read_csv("/content/drive/MyDrive/Volumetric_features.csv")
df.head()

,S.No,Left-Lateral-Ventricle,Left-Inf-Lat-Vent,Left-Cerebellum-White-Matter,Left-Cerebellum-Cortex,Left-Thalamus,Left-Caudate,Left-Putamen,Left-Pallidum,3rd-Ventricle,...,rh_supramarginal_thickness,rh_frontalpole_thickness,rh_temporalpole_thickness,rh_transversetemporal_thickness,rh_insula_thickness,rh_MeanThickness_thickness,BrainSegVolNotVent.2,eTIV.1,Age,dataset
0,1,22916.9,982.7,15196.7,55796.4,6855.5,2956.4,4240.7,2223.9,2034.4,...,2.408,2.629,3.519,2.009,2.825,2.33635,1093846,1619602.965,85,1
1,2,22953.2,984.5,15289.7,55778.6,6835.1,3064.2,4498.6,2354.1,1927.1,...,2.417,2.640,3.488,2.111,2.720,2.34202,1099876,1624755.130,85,1
2,3,23320.4,1062.1,15382.1,55551.2,7566.0,3231.7,4456.2,1995.4,2064.7,...,2.374,2.601,3.342,2.146,2.684,2.31982,1097999,1622609.518,86,1
3,4,24360.0,1000.5,14805.4,54041.8,8004.6,3137.3,4262.2,1983.4,2017.7,...,2.366,2.639,3.361,2.056,2.700,2.29215,1070117,1583854.236,87,1
4,5,25769.4,1124.4,16331.1,54108.6,6677.4,2964.4,4204.6,2409.7,2251.8,...,2.381,2.555,3.450,2.052,2.574,2.30397,1075926,1617375.362,89,1


In [112]:
def preprocessing(standardize=False, normalize=False):
  df = pd.read_csv("/content/drive/MyDrive/Volumetric_features.csv")
  # Cleaning
  df = df.drop(columns=["S.No", "dataset", "Left-WM-hypointensities", "Right-WM-hypointensities", "Left-non-WM-hypointensities", "Right-non-WM-hypointensities"])
  df = df.drop_duplicates()
  df = df.dropna()
  # Split
  y = df["Age"]
  x = df.loc[:,df.columns!="Age"]
  if standardize:
    x=(x-x.mean())/x.std()
  if normalize:
    x=(x-x.min())/(x.max()-x.min())

  return train_test_split(x,y, test_size=0.2, random_state=142)

In [113]:
def process(regressor, std_nrm = True):
  results = {}
  x_train, x_test, y_train, y_test = preprocessing()
  regressor.fit(x_train, y_train)
  results["None"] =   regressor.score(x_test, y_test)
  if not std_nrm:
    return results["None"] 

  x_train, x_test, y_train, y_test = preprocessing(standardize=True)
  regressor.fit(x_train, y_train)
  results["Standardized"] =   regressor.score(x_test, y_test)
  
  x_train, x_test, y_train, y_test = preprocessing(normalize=True)
  regressor.fit(x_train, y_train)
  results["Normalized"] =   regressor.score(x_test, y_test)
  y_pred = regressor.predict(x_test)
  
  print(f"MSE: {mean_squared_error(y_test, y_pred)}")
  return results

## DECISION TREE REGRESSION

In [114]:
from sklearn.tree import DecisionTreeRegressor
DTReg = DecisionTreeRegressor().fit(x_train, y_train)
process(DecisionTreeRegressor())
print("R2: ", DTReg.score(x_test, y_test))


MSE: 129.26004728132386
R2:  0.6797787126764244


## K NEIGHBORS REGRESSION


In [115]:
from sklearn.neighbors import KNeighborsRegressor
KNReg = KNeighborsRegressor().fit(x_train, y_train)
process(KNeighborsRegressor())
print("R2: ", KNReg.score(x_test, y_test))

MSE: 62.28061465721041
R2:  0.7105828079075263
